This is a fantastic example because it forces you to stop thinking of State as a "variable" and start thinking of it as a "stream" of data.

To make this work across multiple "Turns," we need one extra ingredient we haven't used yet: Persistence (Memory). We will use MemorySaver to simulate a database that remembers the state between user interactions.
The Implementation

Here is the complete code. I have annotated it heavily to explain the "Reducer" magic.


In [1]:
import operator
from typing import Annotated, TypedDict

# --- 1. THE STATE DEFINITION ---
class StatsState(TypedDict):
    # CONCEPT: REDUCERS
    # By default, LangGraph overwrites data.
    # Annotated[list, operator.add] tells LangGraph: 
    # "When you get new data for 'scores', DO NOT overwrite. APPEND it."
    scores: Annotated[list[int], operator.add]
    
    # Standard field (Overwrites by default)
    average: float

In [2]:
# --- 2. THE NODE (Logic) ---
def stats_keeper_node(state: StatsState):
    # Step A: Get the full history (The Graph has already appended the new input!)
    current_scores = state["scores"]
    
    # Step B: Calculate the new average
    if not current_scores:
        avg = 0.0
    else:
        avg = sum(current_scores) / len(current_scores)
    
    # Step C: Return ONLY the update for the average
    # We don't need to return 'scores' because they are already in the state.
    print(f"   [Internal Logic] List is now {current_scores}. New Average: {avg}")
    return {"average": avg}

In [3]:
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

# --- 3. THE GRAPH CONSTRUCTION ---
builder = StateGraph(StatsState)

builder.add_node("stats_keeper", stats_keeper_node)
builder.add_edge(START, "stats_keeper")
builder.add_edge("stats_keeper", END)

# CRITICAL: Add a Checkpointer (In-memory database)
# This allows the graph to "remember" previous interactions.
memory = MemorySaver()
app = builder.compile(checkpointer=memory)

In [4]:
# --- 4. THE EXECUTION (Simulating 3 Turns) ---

# We use a thread_id to identify this specific user's session
config = {"configurable": {"thread_id": "student_123"}}

print("--- TURN 1 (Input: 90) ---")
# User sends {"scores": [90]}. 
# Because of 'operator.add', this [90] is APPENDED to the empty list [].
output1 = app.invoke({"scores": [90]}, config=config)
print(f"Final State: {output1}\n")

print("--- TURN 2 (Input: 80) ---")
# User sends {"scores": [80]}. 
# This [80] is APPENDED to the existing [90]. List becomes [90, 80].
output2 = app.invoke({"scores": [80]}, config=config)
print(f"Final State: {output2}\n")

print("--- TURN 3 (Input: 100) ---")
# User sends {"scores": [100]}. 
# This [100] is APPENDED to [90, 80]. List becomes [90, 80, 100].
output3 = app.invoke({"scores": [100]}, config=config)
print(f"Final State: {output3}\n")

--- TURN 1 (Input: 90) ---
   [Internal Logic] List is now [90]. New Average: 90.0
Final State: {'scores': [90], 'average': 90.0}

--- TURN 2 (Input: 80) ---
   [Internal Logic] List is now [90, 80]. New Average: 85.0
Final State: {'scores': [90, 80], 'average': 85.0}

--- TURN 3 (Input: 100) ---
   [Internal Logic] List is now [90, 80, 100]. New Average: 90.0
Final State: {'scores': [90, 80, 100], 'average': 90.0}

